In [1]:
%run "Feature_Selection.ipynb" 

100%|███████████████████████████████████████████████████████████████████████████████| 221/221 [00:01<00:00, 158.40it/s]


Calculating Time Domain Features
Calculating Frequency Domain Features
Calculating Non-Linear Domain Features


<ipython-input-1-b73415202eb2>:10: RuntimeWarning: divide by zero encountered in log
  return -np.log((_phi(m + 1)) / (_phi(m)))
<ipython-input-1-b73415202eb2>:10: RuntimeWarning: divide by zero encountered in log
  return -np.log((_phi(m + 1)) / (_phi(m)))
<ipython-input-1-b73415202eb2>:10: RuntimeWarning: divide by zero encountered in log
  return -np.log((_phi(m + 1)) / (_phi(m)))
<ipython-input-1-b73415202eb2>:10: RuntimeWarning: divide by zero encountered in log
  return -np.log((_phi(m + 1)) / (_phi(m)))
<ipython-input-1-b73415202eb2>:10: RuntimeWarning: divide by zero encountered in log
  return -np.log((_phi(m + 1)) / (_phi(m)))
<ipython-input-1-b73415202eb2>:10: RuntimeWarning: divide by zero encountered in log
  return -np.log((_phi(m + 1)) / (_phi(m)))
<ipython-input-1-b73415202eb2>:10: RuntimeWarning: divide by zero encountered in log
  return -np.log((_phi(m + 1)) / (_phi(m)))
<ipython-input-1-b73415202eb2>:10: RuntimeWarning: divide by zero encountered in log
  return -np

(202, 17)
dict_keys(['rr_amps', 'hf', 'shannon_en', 'age'])
dict_keys(['lf', 'total_power', 'shannon_en', 'age'])
dict_keys(['rr_amps', 'kurtosis', 'shannon_en', 'age'])
dict_keys(['rr_std', 'lf', 'total_power', 'age'])
dict_keys(['rr_amps', 'std', 'skews', 'age'])
dict_keys(['rr_std', 'hf', 'lf', 'age'])


In [2]:
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, balanced_accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import itertools

In [3]:
# my_params = [['kurtosis', 'sd_ratio', 'lf'], ['sd_ratio_outliers_removed', 'shannon_en', 'pNN50'], ['pNN50', 'shannon_en', 'sd_ratio_outliers_removed'], ['pNN50', 'lf', 'sd_ratio_outliers_removed'], ['lf', 'pNN50', 'shannon_en'], ['lf', 'shannon_en', 'hf']]
        
# selected_params = {}
# for i in range(no_channels):
#     selected_params[i] = {}  # Initialize as an empty dictionary
#     for param in my_params[i]:
#         selected_params[i][param] = params[param][i]

In [4]:
health_state = np.array(allowed_patients.get_diagnoses())

encoded_health_state = [1 if label == 'Unhealthy' else -1 for label in health_state]

In [5]:
def get_av_confusion_matrix(y_test, y_pred):
    av_confusion_mat = np.zeros(shape = (len(y_test), 2, 2))
    for i in range(0, len(y_test)):
        av_confusion_mat[i] = confusion_matrix(y_test[i], y_pred[i])
    return np.mean(av_confusion_mat, axis=0)

def objective_score(y_test, y_pred):
    """
    change to incorporate balanced accuracy 
    """
    balanced_acc = get_balanced_accuracy(y_test, y_pred)
    specificity = get_specificity(y_test, y_pred)
    f1 = get_f1_score(y_test, y_pred)
    
    
    return f1*0.7 + balanced_acc*0.3

def convert_multi_dict_to_array1(params_dict, nan_indices):
    no_features = len(params_dict[0])
    params_list = []
    for j in range(0, no_channels):
        params_array = np.zeros((len(health_state[nan_indices[j]]), no_features))
        for i, values in enumerate(params_dict[j].values()):
            params_array[:, i] = values
        params_list.append(params_array)
    return params_list, no_features





First convert the dictionary containing the parameters to use for each channel into an array of suitable shape for svm models. 

In [6]:
print(len(selected_params[0]))

4


In [7]:
selected_params_array, no_features = convert_multi_dict_to_array1(selected_params, nan_indices)

In [8]:
def tune_hyperparams(params, health_state, param_grid, scorer='balanced_accuracy'):
    
    X_train, X_test, y_train, y_test = train_test_split(params, health_state, test_size=0.3, stratify=health_state)
    
    #initialise classifier
    svc = SVC(class_weight='balanced', probability = True)

    # perform grid search
    grid_search = GridSearchCV(svc, param_grid, cv=3, scoring=scorer)
    
    grid_search.fit(X_train, y_train)
    
    return grid_search.best_estimator_

Then we are optimizing the hyperparameters that are used to train the model by perfroma grid search over the ``param_grid`` dictionary. The hyperparameters are chosen based on their optimisation of the manually defined ``scoring_function``. This takes a while as it has to perform a search over all the possible hyperparameters for each channel.

In [9]:
# define hyperparameter grid to test
param_grid = {
    'C': [0.01, 0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale']#including 'auto' aswell takes forever
}


best_estimators = []
for i in range(0, no_channels):
    best_estimators.append(tune_hyperparams(selected_params_array[i], health_state[nan_indices[i]], param_grid, scoring_function))
    

In [10]:
print(best_estimators)

[SVC(C=10, class_weight='balanced', probability=True), SVC(C=0.1, class_weight='balanced', probability=True), SVC(C=10, class_weight='balanced', probability=True), SVC(C=10, class_weight='balanced', probability=True), SVC(C=10, class_weight='balanced', probability=True), SVC(C=1, class_weight='balanced', probability=True)]


Now using the best set of hyperparameters we perform a 3 fold stratisfied split to investigate the accuracy. Firstly we can investigate the average accuracy of each channel.

In [11]:
def skfold_with_probabilities(params, health_state, n_splits, best_estimator, scorer):
    
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42) #can do repeated skf for better validation
    
    probabilities = []
    sample_percentages = []
    y_tests = []
    balanced_accuracy = []
    score_func = []
    test_indices = []
    for train_index, test_index in skf.split(params, health_state):
        #getting test and train data sets
        X_train, X_test = params[train_index], params[test_index]
        y_train, y_test = health_state[train_index], health_state[test_index]
        
        
        #calculating percentage of healthy in training data
        sample_percentages.append(np.mean(y_train == 'Healthy')) #how is threshold measured??? health_state[train_index]??
        
        #fitting data on previously calculated best estimator
        best_estimator.fit(X_train, y_train)
        
        #evaluating model
        y_pred = best_estimator.predict(X_test)
        score_func.append(scorer(best_estimator, X_train, y_test))
        balanced_accuracy.append(get_balanced_accuracy(y_test, y_pred))
        
        #for evaluation of model later
        probabilities.append(best_estimator.predict_proba(X_test))
        y_tests.append(y_test)
        
        #for reconstruction of full patient data
        test_indices.append(test_index)

    return np.mean(np.array(score_func)), np.mean(np.array(balanced_accuracy)), probabilities, sample_percentages, y_tests, test_indices

In [12]:
score_accuracy = []
balanced_accuracy = []
n_splits=3

probs = []
thresholds = []
y_tests = []
test_indices = []

for i in range(0, no_channels):
    score_acc, bal_acc, prob, threshold, y_test, test_indice = skfold_with_probabilities(selected_params_array[i], health_state[nan_indices[i]], n_splits, best_estimators[i], scoring_function)
    score_accuracy.append(score_acc)
    balanced_accuracy.append(bal_acc)
    probs.append(prob)
    thresholds.append(threshold)
    y_tests.append(y_test)
    test_indices.append(test_indice)

In [13]:
import pandas as pd

#presenting results as pandas df
data = {
    'Success Metric': ['Objective Score', 'Balanced Accuracy'],
    'Channel 1': [f'{score_accuracy[0]}', f'{balanced_accuracy[0]}'],
    'Channel 2': [f'{score_accuracy[1]}', f'{balanced_accuracy[1]}'],
    'Channel 3': [f'{score_accuracy[2]}', f'{balanced_accuracy[2]}'],
    'Channel 4': [f'{score_accuracy[3]}', f'{balanced_accuracy[3]}'],
    'Channel 5': [f'{score_accuracy[4]}', f'{balanced_accuracy[4]}'],
    'Channel 6': [f'{score_accuracy[5]}', f'{balanced_accuracy[5]}'],
    
}

df = pd.DataFrame(data)

df

,Success Metric,Channel 1,Channel 2,Channel 3,Channel 4,Channel 5,Channel 6
0,Objective Score,0.7239246218080847,0.7352390132389237,0.7632646339168078,0.7222567851256564,0.7236904638628777,0.7387254249361167
1,Balanced Accuracy,0.6901960784313724,0.6907469342251952,0.6957070707070706,0.6530159633767404,0.7004409171075837,0.6987363789250581


Then can average the probabilities for each channel to get an overall accuracy for the method.

As the arrays are now all different length with different patients' date having been discarded the below no longer works. will try to recreate each patients data from the array indices and test indices given by sklearn so that probabilities can be compared.

In [14]:
def reconstruct_probs(probs, test_indices, nan_indices):
    reconstructed_probs = np.zeros(no_patients)
    reconstructed_probs[~nan_indices] = np.nan
    allowed_indices = np.arange(0, no_patients)[nan_indices]

    for z in range(0, n_splits):
        for i, indice in enumerate(test_indices[z]):
            reconstructed_probs[allowed_indices[indice]] = probs[z][i][0]
            

    return reconstructed_probs
        
    

In [15]:
reconstructed_probs = []
for i in range(0, no_channels):
    reconstructed_prob = reconstruct_probs(probs[i], test_indices[i], nan_indices[i])
    reconstructed_probs.append(reconstructed_prob)

In [16]:
# def average_probabilities(probs, n_splits, channels):
#     average_probs = []
#     for i in range(0, n_splits):
#         probas = []
#         for channel in channels:
#             probas.append(probs[channel][i])
#         average_probs.append(np.mean(probas, axis=0)[:, 0])
#     return average_probs

# def manual_y_pred(average_probs, n_splits, thresholds):
#     manual_y_pred = []
#     for i in range(0, n_splits):
#         manual_predict = []
#         for j in range(0, len(average_probs[i])):
#             if average_probs[i][j] > thresholds[0][i]:
#                 manual_predict.append('Healthy')
#             else:
#                 manual_predict.append('Unhealthy')
#         manual_y_pred.append(manual_predict)
#     return manual_y_pred

def average_probabilities(probs, channels):
    average_probs = []
    for channel in channels:
        average_probs.append(probs[channel])

    return np.nanmean(average_probs, axis=0)
    
def manual_y_predict(average_probs, threshold):
    manual_y_pred = np.empty(len(average_probs), dtype=object)
    for j in range(0, len(average_probs)):
        if average_probs[j] > 0.3:
            manual_y_pred[j] = 'Healthy'
        elif average_probs[j] <= 0.3:
            manual_y_pred[j] = 'Unhealthy'
        elif average_probs[j] is None:
            manual_y_pred[j] = np.nan
    return manual_y_pred



channel_indices_list = [0, 1, 2, 3, 4, 5]
all_combinations = []
for r in range(1, len(probs) + 1):
    combination = list(itertools.combinations(channel_indices_list, r))
    all_combinations.extend(combination)
    
#print(thresholds)
best_score = 0
best_channel_indices = []
for combo in all_combinations:
        
        # Use combo to get channel indices
        selected_channel_indices = [channel_indices_list[i] for i in combo]
        
        #print(selected_channel_indices)
        #no longer have any splits
        #combo_probs = average_probabilities(probs, n_splits, selected_channel_indices)
        #manual_pred = manual_y_pred(combo_probs, n_splits, thresholds)
        
        combo_probs = average_probabilities(reconstructed_probs, selected_channel_indices)
        manual_pred = manual_y_predict(combo_probs, thresholds)


        #dont have any channels, is all combined into one
        score = objective_score(health_state, manual_pred)
            
        if score > best_score:
            best_score = score
            best_channel_indices = selected_channel_indices


<ipython-input-16-5ab2b991886d>:27: RuntimeWarning: Mean of empty slice
  return np.nanmean(average_probs, axis=0)


In [17]:
print(best_score)
print(best_channel_indices)

0.8448497069372758
[1, 2, 3, 4, 5]


In [18]:
for i in range(0, no_channels):
    print(selected_params[i].keys())

dict_keys(['rr_amps', 'hf', 'shannon_en', 'age'])
dict_keys(['lf', 'total_power', 'shannon_en', 'age'])
dict_keys(['rr_amps', 'kurtosis', 'shannon_en', 'age'])
dict_keys(['rr_std', 'lf', 'total_power', 'age'])
dict_keys(['rr_amps', 'std', 'skews', 'age'])
dict_keys(['rr_std', 'hf', 'lf', 'age'])
